In [2]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)
import numpy as np
import pandas as pd

items = pd.read_csv(r"..\Data\Raw\items.csv", delimiter=",")
print(items.head())
print(items.shape)
items = items.drop_duplicates(subset=['item_name'], keep=False)
print(items.shape)

                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id  
0                40  
1                76  
2                40  
3                40  
4                40  
(22170, 3)
(22170, 3)


In [3]:
test = pd.read_csv(r"..\Data\Raw\test.csv", delimiter=",")
print(test.head())
print(test['shop_id'].nunique())
print(test['ID'].nunique())

   ID  shop_id  item_id
0   0        5     5037
1   1        5     5320
2   2        5     5233
3   3        5     5232
4   4        5     5268
42
214200


In [4]:
sales = pd.read_csv(r"..\Data\Raw\sales_train.csv", delimiter=",")
print(sales.head())
print(sales.shape)
sales = sales.drop_duplicates(subset=['date', 'shop_id', 'item_id'], keep=False)
print(sales.shape)
sales = sales[sales.item_cnt_day<1000]
sales = sales[sales.item_price<60000]
sales = sales[sales.item_price>0]
print(sales.shape)
sales['revenue'] = sales['item_cnt_day'] * sales['item_price']
sales_monthly = sales.groupby(["date_block_num", "shop_id", "item_id"]).agg({'item_cnt_day': ['sum'], 'item_price':['median'], 'revenue':['sum']})
print(sales_monthly.head(20))
sales_monthly.sample(10)

         date  date_block_num  shop_id  item_id  item_price  item_cnt_day
0  02.01.2013               0       59    22154      999.00           1.0
1  03.01.2013               0       25     2552      899.00           1.0
2  05.01.2013               0       25     2552      899.00          -1.0
3  06.01.2013               0       25     2554     1709.05           1.0
4  15.01.2013               0       25     2555     1099.00           1.0
(2935849, 6)
(2935793, 6)
(2935789, 6)
                               item_cnt_day item_price revenue
                                        sum     median     sum
date_block_num shop_id item_id                                
0              0       32               6.0      221.0  1326.0
                       33               3.0      347.0  1041.0
                       35               1.0      247.0   247.0
                       43               1.0      221.0   221.0
                       51               2.0      128.5   257.0
             

item_cnt_day item_price revenue
                                        sum     median     sum
date_block_num shop_id item_id                                
0              19      17412            1.0     149.00   149.0
8              54      4227             1.0     899.00   899.0
               42      16333            1.0     349.00   349.0
6              42      3606             2.0     299.00   598.0
5              4       19784            1.0     299.00   299.0
19             31      4910             2.0     418.85   837.7
12             25      13890            4.0     399.00  1596.0
5              56      15583            1.0     349.00   349.0
23             45      7203             1.0     199.00   199.0
16             22      21645            1.0     199.00   199.0

In [5]:
#join to test set to get only necessary IDs
month_range = range(1, 34)
month_df = pd.DataFrame(month_range)

month_df.rename(columns={0 :'date_block_num'}, inplace=True )
print(month_df.head())
month_df['temp'] = 1
test['temp'] = 1
test_monthly = test.merge(month_df, on=['temp'])
test_monthly.drop(['temp'], axis=1, inplace=True)
whole_df = test_monthly.merge(sales_monthly, how='left', on=['date_block_num', 'item_id', 'shop_id'])
whole_df.sample(10)
print(whole_df.shape)

   date_block_num
0               1
1               2
2               3
3               4
4               5
(7068600, 7)


e:\latest pycharm\venv\lib\site-packages\pandas\core\reshape\merge.py:643: UserWarning: merging between different levels can give an unintended result (1 levels on the left,2 on the right)
  warnings.warn(msg, UserWarning)


In [6]:
categories = pd.read_csv(r"..\Data\Raw\item_categories.csv", delimiter=",")
print(categories.head())


        item_category_name  item_category_id
0  PC - Гарнитуры/Наушники                 0
1         Аксессуары - PS2                 1
2         Аксессуары - PS3                 2
3         Аксессуары - PS4                 3
4         Аксессуары - PSP                 4


In [7]:
print(items.shape)
print(categories.shape)
item_cats = items.merge(categories, how='left', on=['item_category_id'])
print(item_cats.shape)
print(item_cats.head())
item_cats.drop(['item_category_name', 'item_name'], axis=1, inplace=True)
print(item_cats.head())
print(item_cats['item_id'].nunique())
print(item_cats.shape)
whole_df = whole_df.merge(item_cats, how='left', on=['item_id'])
whole_df.sample(10)

(22170, 3)
(84, 2)
(22170, 4)
                                           item_name  item_id  \
0          ! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.)         D        0   
1  !ABBYY FineReader 12 Professional Edition Full...        1   
2      ***В ЛУЧАХ СЛАВЫ   (UNV)                    D        2   
3    ***ГОЛУБАЯ ВОЛНА  (Univ)                      D        3   
4        ***КОРОБКА (СТЕКЛО)                       D        4   

   item_category_id                    item_category_name  
0                40                            Кино - DVD  
1                76  Программы - Для дома и офиса (Цифра)  
2                40                            Кино - DVD  
3                40                            Кино - DVD  
4                40                            Кино - DVD  
   item_id  item_category_id
0        0                40
1        1                76
2        2                40
3        3                40
4        4                40
22170
(22170, 2)


,ID,shop_id,item_id,date_block_num,"(item_cnt_day, sum)","(item_price, median)","(revenue, sum)",item_category_id
3391848,102783,42,13279,10,NaN,NaN,NaN,47
6056170,183520,38,16057,11,2.0,619.0,1238.0,64
6814743,206507,39,3640,13,NaN,NaN,NaN,55
6008211,182067,38,9457,1,NaN,NaN,NaN,57
3126833,94752,14,2708,18,NaN,NaN,NaN,23
4593234,139188,57,17108,31,1.0,249.0,249.0,40
1278632,38746,12,3377,15,NaN,NaN,NaN,58
5858795,177539,35,21980,9,NaN,NaN,NaN,57
1603530,48591,31,18714,28,NaN,NaN,NaN,40
1511191,45793,28,9596,23,NaN,NaN,NaN,40


In [8]:
whole_df.fillna(0, inplace=True)
list(whole_df.columns)
whole_df.rename(columns={('item_cnt_day', 'sum'):'total_count', ('item_price', 'median'): 'median_price', ('revenue', 'sum'):'total_revenue'}, inplace=True )
whole_df['total_count'].clip(0,20)
print(whole_df.sample(10))
whole_df.to_csv('../Data/Processed/pre_normalised_whole_df.csv', index=False)

             ID  shop_id  item_id  date_block_num  total_count  median_price  \
5092833  154328       55    21676              10          0.0           0.0   
2908441   88134       18    12117              20          0.0           0.0   
2964403   89830       18    19337              14          0.0           0.0   
6399434  193922       41     9398               9          0.0           0.0   
4415060  133789       48     8397              24          0.0           0.0   
1236711   37476       12    12232               4          2.0         350.0   
2883854   87389       18    13558              18          0.0           0.0   
2359995   71515       21     6062               1          0.0           0.0   
6808231  206310       39     9171               2          0.0           0.0   
6582380  199466       44     2614               3          0.0           0.0   

         total_revenue  item_category_id  
5092833            0.0                40  
2908441            0.0           

In [9]:
whole_df.sort_values(by=['ID', 'date_block_num'])
print(whole_df.head(34))
whole_df = whole_df[whole_df['ID'] == 46360 or ]
whole_df.drop(['ID', 'shop_id', 'item_id', 'median_price', 'total_revenue', 'item_category_id'], axis=1, inplace=True)
whole_df_arr = whole_df.values
print(whole_df_arr.shape)
whole_df_arr = whole_df_arr.reshape(2, 33, 2)
print(whole_df_arr)#change back to 214200

    ID  shop_id  item_id  date_block_num  total_count  median_price  \
0    0        5     5037               1          0.0           0.0   
1    0        5     5037               2          0.0           0.0   
2    0        5     5037               3          0.0           0.0   
3    0        5     5037               4          0.0           0.0   
4    0        5     5037               5          0.0           0.0   
5    0        5     5037               6          0.0           0.0   
6    0        5     5037               7          0.0           0.0   
7    0        5     5037               8          0.0           0.0   
8    0        5     5037               9          0.0           0.0   
9    0        5     5037              10          0.0           0.0   
10   0        5     5037              11          0.0           0.0   
11   0        5     5037              12          0.0           0.0   
12   0        5     5037              13          0.0           0.0   
13   0

In [10]:
print(whole_df_arr)

[[[1.000e+00 0.000e+00]
  [2.000e+00 0.000e+00]
  [3.000e+00 1.300e+01]
  [4.000e+00 3.840e+02]
  [5.000e+00 7.630e+02]
  [6.000e+00 7.990e+02]
  [7.000e+00 8.200e+02]
  [8.000e+00 9.500e+02]
  [9.000e+00 9.780e+02]
  [1.000e+01 9.890e+02]
  [1.100e+01 1.305e+03]
  [1.200e+01 8.990e+02]
  [1.300e+01 9.410e+02]
  [1.400e+01 7.760e+02]
  [1.500e+01 5.970e+02]
  [1.600e+01 6.020e+02]
  [1.700e+01 6.250e+02]
  [1.800e+01 5.280e+02]
  [1.900e+01 5.910e+02]
  [2.000e+01 6.390e+02]
  [2.100e+01 6.340e+02]
  [2.200e+01 7.720e+02]
  [2.300e+01 1.209e+03]
  [2.400e+01 7.430e+02]
  [2.500e+01 1.800e+02]
  [2.600e+01 0.000e+00]
  [2.700e+01 4.810e+02]
  [2.800e+01 4.600e+02]
  [2.900e+01 4.340e+02]
  [3.000e+01 4.820e+02]
  [3.100e+01 4.360e+02]
  [3.200e+01 4.730e+02]
  [3.300e+01 4.310e+02]]]


In [11]:
#prepare training and validation sets
train_arr = whole_df_arr[:, :26]
val_arr = whole_df_arr[:, 26:]
print(train_arr.shape)
print(val_arr.shape)
train_arr_first_test = train_arr[:, 19:25]
print(train_arr_first_test.shape)
print(train_arr)

(1, 26, 2)
(1, 7, 2)
(1, 6, 2)
[[[1.000e+00 0.000e+00]
  [2.000e+00 0.000e+00]
  [3.000e+00 1.300e+01]
  [4.000e+00 3.840e+02]
  [5.000e+00 7.630e+02]
  [6.000e+00 7.990e+02]
  [7.000e+00 8.200e+02]
  [8.000e+00 9.500e+02]
  [9.000e+00 9.780e+02]
  [1.000e+01 9.890e+02]
  [1.100e+01 1.305e+03]
  [1.200e+01 8.990e+02]
  [1.300e+01 9.410e+02]
  [1.400e+01 7.760e+02]
  [1.500e+01 5.970e+02]
  [1.600e+01 6.020e+02]
  [1.700e+01 6.250e+02]
  [1.800e+01 5.280e+02]
  [1.900e+01 5.910e+02]
  [2.000e+01 6.390e+02]
  [2.100e+01 6.340e+02]
  [2.200e+01 7.720e+02]
  [2.300e+01 1.209e+03]
  [2.400e+01 7.430e+02]
  [2.500e+01 1.800e+02]
  [2.600e+01 0.000e+00]]]


In [12]:
#normalise data

train_mean = train_arr.mean(axis=(0, 1))
train_std = train_arr.std(axis=(0, 1))
train_arr = (train_arr - train_mean)/train_std
print(train_arr)
val_arr = (val_arr - train_mean)/train_std


[[[-1.66666667 -1.808996  ]
  [-1.53333333 -1.808996  ]
  [-1.4        -1.77246372]
  [-1.26666667 -0.72988887]
  [-1.13333333  0.33516739]
  [-1.          0.43633368]
  [-0.86666667  0.49534735]
  [-0.73333333  0.86067008]
  [-0.6         0.93935497]
  [-0.46666667  0.9702669 ]
  [-0.33333333  1.85828214]
  [-0.2         0.71735116]
  [-0.06666667  0.83537851]
  [ 0.06666667  0.37169966]
  [ 0.2        -0.13132163]
  [ 0.33333333 -0.11727076]
  [ 0.46666667 -0.05263674]
  [ 0.6        -0.32522369]
  [ 0.73333333 -0.14818268]
  [ 0.86666667 -0.01329429]
  [ 1.         -0.02734516]
  [ 1.13333333  0.36045896]
  [ 1.26666667  1.58850536]
  [ 1.4         0.27896389]
  [ 1.53333333 -1.30316453]
  [ 1.66666667 -1.808996  ]]]


In [13]:
#train_arr = list(train_arr)

#train_arr = train_arr[0:3]
train_list = list(train_arr)
val_list = list(val_arr)
first_test_list = list(train_arr_first_test)
#def split_sequences(sequences, n_steps):
#    pass

In [14]:
window_size = 6

def windowed_dataset(array, window_size):
    ds = tf.data.Dataset.from_tensor_slices(array)
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda x: x.batch(window_size + 1))
    ds = ds.map(lambda x: (x[:-1], x[-1][1]))
    return ds
#dataset = windowed_dataset(train_arr, window_size, batch_size)
#for d in dataset:
#   print(d)

In [15]:
print(train_arr.shape)
dataset = tf.data.Dataset.from_generator(lambda: train_list, tf.as_dtype(train_list[0].dtype), tf.TensorShape([26,2]))
dataset = dataset.flat_map(lambda x: windowed_dataset(x, window_size))
dataset = dataset.batch(1).prefetch(1)
for i in dataset:
    print(i)

(1, 26, 2)
(<tf.Tensor: shape=(1, 6, 2), dtype=float64, numpy=
array([[[-1.66666667, -1.808996  ],
        [-1.53333333, -1.808996  ],
        [-1.4       , -1.77246372],
        [-1.26666667, -0.72988887],
        [-1.13333333,  0.33516739],
        [-1.        ,  0.43633368]]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.49534735])>)
(<tf.Tensor: shape=(1, 6, 2), dtype=float64, numpy=
array([[[-1.53333333, -1.808996  ],
        [-1.4       , -1.77246372],
        [-1.26666667, -0.72988887],
        [-1.13333333,  0.33516739],
        [-1.        ,  0.43633368],
        [-0.86666667,  0.49534735]]])>, <tf.Tensor: shape=(1,), dtype=float64, numpy=array([0.86067008])>)
(<tf.Tensor: shape=(1, 6, 2), dtype=float64, numpy=
array([[[-1.4       , -1.77246372],
        [-1.26666667, -0.72988887],
        [-1.13333333,  0.33516739],
        [-1.        ,  0.43633368],
        [-0.86666667,  0.49534735],
        [-0.73333333,  0.86067008]]])>, <tf.Tensor: shape=(1,), dtype=float64, 

In [16]:
val_dataset = tf.data.Dataset.from_generator(lambda: val_list, tf.as_dtype(val_list[0].dtype), tf.TensorShape([7,2]))
val_dataset = val_dataset.flat_map(lambda x: windowed_dataset(x, window_size))
val_dataset = val_dataset.batch(1).prefetch(1)

In [17]:
sequences = [np.arange(5)[:, None, None], np.arange(20, 24)[:, None, None]]
for i in sequences:
    print(i.shape)
#for i in train_list:
#    print(i.shape)

(5, 1, 1)
(4, 1, 1)


In [18]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True, input_shape=(6,2))),
  tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer="adam")
history = model.fit(dataset,validation_data=val_dataset, epochs=200,verbose=1)

Epoch 1/200

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

20/20 [==============================] - 1s 69ms/step - loss: 0.6344 - val_loss: 0.3524
Epoch 2/200
20/20 [==============================] - 0s 7ms/step - loss: 0.6409 - val_loss: 0.0072
Epoch 3/200
20/20 [==============================] - 0s 7ms/step - loss: 0.4790 - val_loss: 0.0056

In [24]:
print(val_arr)
#create prediction window excluding last value to compare
predictions = model.predict(val_arr[:,:-1])
denormalised_predictions = predictions*train_std[1] + train_mean[1]

[[[ 1.8        -0.45730191]
  [ 1.93333333 -0.51631558]
  [ 2.06666667 -0.58938013]
  [ 2.2        -0.45449174]
  [ 2.33333333 -0.58375978]
  [ 2.46666667 -0.47978331]
  [ 2.6        -0.59781065]]]


In [25]:
print(denormalised_predictions)

[[848.59863]]


In [21]:
rando = np.arange(0, 416).reshape([2,26,8])
print(rando)
rando_mean = rando.mean(axis=(0,1))
print(rando_mean)
rando_mean = rando.mean(axis=(0))
print(rando_mean)

[[[  0   1   2   3   4   5   6   7]
  [  8   9  10  11  12  13  14  15]
  [ 16  17  18  19  20  21  22  23]
  [ 24  25  26  27  28  29  30  31]
  [ 32  33  34  35  36  37  38  39]
  [ 40  41  42  43  44  45  46  47]
  [ 48  49  50  51  52  53  54  55]
  [ 56  57  58  59  60  61  62  63]
  [ 64  65  66  67  68  69  70  71]
  [ 72  73  74  75  76  77  78  79]
  [ 80  81  82  83  84  85  86  87]
  [ 88  89  90  91  92  93  94  95]
  [ 96  97  98  99 100 101 102 103]
  [104 105 106 107 108 109 110 111]
  [112 113 114 115 116 117 118 119]
  [120 121 122 123 124 125 126 127]
  [128 129 130 131 132 133 134 135]
  [136 137 138 139 140 141 142 143]
  [144 145 146 147 148 149 150 151]
  [152 153 154 155 156 157 158 159]
  [160 161 162 163 164 165 166 167]
  [168 169 170 171 172 173 174 175]
  [176 177 178 179 180 181 182 183]
  [184 185 186 187 188 189 190 191]
  [192 193 194 195 196 197 198 199]
  [200 201 202 203 204 205 206 207]]

 [[208 209 210 211 212 213 214 215]
  [216 217 218 219 220 221

In [22]:
rando = np.arange(0, 10).reshape([2,5])
print(rando)
print(rando.mean())
rando_mean = rando.mean(axis=(0,1))
print(rando_mean)

[[0 1 2 3 4]
 [5 6 7 8 9]]
4.5
4.5


In [23]:
rando = np.arange(0, 24).reshape([2,3,4])
print(rando)

rando_mean = rando.mean(axis=(0,1))
print(rando_mean)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]
[10. 11. 12. 13.]
